


# Week 09 Live Coding Demo: Linear Algebra I (Ax=b, Conditioning, Least Squares)

**Flow**: vectors & dot products → matrices as linear maps → build Ax=b from physics → elimination intuition → conditioning → multiple RHS → least squares fit for g.

Notes:
- Examples are **different** from the slides, but cover the same knowledge points.
- Code is heavily commented to explain the physics motivation and linear algebra steps.


In [ ]:
# Imports used throughout the notebook
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (6, 3.6)
np.set_printoptions(precision=4, suppress=True)


## 1) Vectors, norms, and projections — component of gravity along an incline

In [ ]:
# Physics motivation:
# On an incline at angle theta, the component of gravitational acceleration g along the slope is g_parallel = g * sin(theta).
# We'll compute it using a dot product with a unit vector along the slope, to connect to vector norms and projections.

g = np.array([0.0, -9.81])                      # gravitational acceleration [m/s^2], downward in y
theta = np.deg2rad(25.0)                        # incline angle
u_slope = np.array([np.cos(theta), np.sin(theta)])  # unit vector along slope (pointing "up the hill")

# Projection of g onto the slope direction (work-like dot product divided by norm^2 which is 1 here)
g_parallel = (g @ u_slope) * u_slope            # vector component along slope
g_scalar_along = g @ u_slope                    # signed magnitude along slope (should be negative: down the slope)

print("unit vector along slope u_slope =", u_slope)          # e.g., [0.9063, 0.4226]
print("||u_slope|| =", np.linalg.norm(u_slope))               # -> 1.0
print("g · u_slope =", round(float(g_scalar_along), 4), "m/s^2")  # -> negative number, magnitude is g*sin(theta)
print("vector component of g along slope =", g_parallel)      # vector projection along slope

# Visualize vectors
plt.figure()
origin = np.array([[0,0]])
plt.quiver(*origin.T, [g[0]],[g[1]], angles='xy', scale_units='xy', scale=1, color='C0', label='g')
plt.quiver(*origin.T, [u_slope[0]],[u_slope[1]], angles='xy', scale_units='xy', scale=1, color='C1', label='u_slope')
plt.quiver(*origin.T, [g_parallel[0]],[g_parallel[1]], angles='xy', scale_units='xy', scale=1, color='C3', label='g∥')
plt.axis('equal'); plt.xlim(-10,10); plt.ylim(-10,2)
plt.grid(True); plt.legend(); plt.title("Projection of g onto slope direction")
plt.show()


## 2) Dot product as instantaneous power — charged particle in uniform electric field

In [ ]:
# Physics motivation:
# Instantaneous power delivered to a particle is P = F · v. For electric force, F = q E in a uniform field.
# We'll simulate a particle with a time-varying velocity and compute P(t) = q E · v(t).

q = 1.6e-19                                      # charge [C]
E = np.array([2.0e4, 1.0e4])                     # uniform electric field [V/m] = [N/C]
t = np.linspace(0, 1e-6, 300)                    # time [s]
# A simple 2D velocity profile (m/s): circular drift + small acceleration
v = np.vstack([ 2e5*np.cos(2*np.pi*2e6*t) + 3e11*t,
                2e5*np.sin(2*np.pi*2e6*t) ]).T

P = q*(v @ E)                                    # P(t) scalar for each time sample
print("P(t) range [min, max] =", float(P.min()), float(P.max()), "W")  # instantaneous power can be negative or positive depending on direction

plt.figure()
plt.plot(t*1e6, P)
plt.xlabel("time [µs]"); plt.ylabel("instantaneous power P [W]")
plt.title("Power P = q E · v(t)")
plt.grid(True); plt.tight_layout(); plt.show()


## 3) Matrices as linear maps — rotate and stretch a detector footprint

In [ ]:
# Motivation: A 2×2 matrix can represent a coordinate transform (e.g., rotate, then anisotropically scale)
# for a detector plane. We'll transform a grid and a unit circle and visualize the effect.

phi = np.deg2rad(30)                 # 30-degree rotation
R = np.array([[np.cos(phi), -np.sin(phi)],
              [np.sin(phi),  np.cos(phi)]])     # rotation
S = np.diag([1.5, 0.8])                          # stretch: x by 1.5, y by 0.8
A = S @ R                                        # first rotate, then stretch

# Build a grid and a unit circle
theta = np.linspace(0, 2*np.pi, 200)
circle = np.vstack([np.cos(theta), np.sin(theta)])  # shape (2,200)

xs = np.linspace(-1,1,11); ys = np.linspace(-1,1,11)
grid_lines = []
for x in xs:
    grid_lines.append(np.vstack([np.full_like(ys, x), ys]))
for y in ys:
    grid_lines.append(np.vstack([xs, np.full_like(xs, y)]))

plt.figure(figsize=(5,5))
# original
for L in grid_lines:
    plt.plot(L[0], L[1], 'k:', alpha=0.25)
plt.plot(circle[0], circle[1], 'k--', lw=1.2, label='unit circle')
# transformed
for L in grid_lines:
    L2 = A @ L
    plt.plot(L2[0], L2[1], 'C0-', alpha=0.7)
circ2 = A @ circle
plt.plot(circ2[0], circ2[1], 'C3-', lw=2, label='transformed circle')
plt.axis('equal'); plt.title("Linear map A = S R on grid and circle")
plt.legend(); plt.tight_layout(); plt.show()

# Verify linearity: A(ax + bz) = a A x + b A z
x = np.array([0.6, -0.4]); z = np.array([-0.2, 0.9]); a, b = 2.0, -0.5
left = A @ (a*x + b*z)
right = a*(A@x) + b*(A@z)
print("Linearity check ||left-right|| =", np.linalg.norm(left-right))  # -> ~0


## 4) (Optional) From physics to Ax=b — 1D rod steady-state heat conduction (Dirichlet BCs)

In [ ]:
# Physics model: steady-state heat conduction (no internal heat generation) in 1D implies d^2 T / dx^2 = 0.
# Discretize the rod [0, L] with interior nodes; enforce boundary temperatures T(0)=T_left, T(L)=T_right.
# The finite-difference approximation at interior node j is: T_{j-1} - 2 T_j + T_{j+1} = 0
# This yields a tridiagonal linear system A T = b.

L = 1.0
N_int = 8                                   # number of interior nodes (unknowns)
h = L / (N_int + 1)                         # uniform spacing
T_left, T_right = 100.0, 20.0               # boundary temperatures [°C]

# Build A (tridiagonal) and b (incorporate BCs)
A = np.zeros((N_int, N_int))
b = np.zeros(N_int)
for j in range(N_int):
    A[j, j] = -2.0
    if j-1 >= 0: A[j, j-1] = 1.0
    if j+1 < N_int: A[j, j+1] = 1.0
b[0] -= T_left
b[-1] -= T_right

T_int = np.linalg.solve(A, b)               # interior temperatures
x = np.linspace(h, L-h, N_int)
T = np.r_[T_left, T_int, T_right]           # include boundaries
x_full = np.r_[0.0, x, L]

print("Interior temperatures:", np.round(T_int, 2))  # monotone linear trend expected for Laplace

plt.figure()
plt.plot(x_full, T, 'o-', label='T(x)')
plt.xlabel("x [m]"); plt.ylabel("Temperature [°C]")
plt.title("Steady 1D rod conduction (Dirichlet BCs)")
plt.grid(True); plt.legend(); plt.tight_layout(); plt.show()


## 5) Gaussian elimination by hand (augmented matrix steps)

In [ ]:
# We'll use a small (different) 3x3 system that could arise from a mass balance in a 3-cell diffusion toy:
# eqs:  3x +  y -  z =  1
#       2x - 2y + 4z = -2
#      -x + 0.5y - z =  0
A = np.array([[ 3.,  1., -1.],
              [ 2., -2.,  4.],
              [-1.,  0.5,-1.]], float)
b = np.array([1., -2., 0.], float)
Ab = np.c_[A,b] # Combines A and b into an augmented matrix by column concatenation
print("Start [A|b]:\n", Ab)

# Eliminate the first column below the pivot (row 0 pivot = 3)
f10 = Ab[1,0]/Ab[0,0]; Ab[1] -= f10*Ab[0]
f20 = Ab[2,0]/Ab[0,0]; Ab[2] -= f20*Ab[0]
print("\nAfter eliminating below row 0 pivot:\n", np.round(Ab,4))

# Eliminate the second column below the pivot (row 1 pivot is Ab[1,1])
pivot = Ab[1,1]
f21 = Ab[2,1]/pivot; Ab[2] -= f21*Ab[1]
print("\nUpper-triangular [U|c]:\n", np.round(Ab,4))

# Back-substitution (solve U x = c)
U = Ab[:,:-1]; c = Ab[:,-1]
x = np.zeros(3)
x[2] = c[2]/U[2,2]
x[1] = (c[1] - U[1,2]*x[2]) / U[1,1]
x[0] = (c[0] - U[0,1]*x[1] - U[0,2]*x[2]) / U[0,0]
print("\nSolution x =", np.round(x, 6))

# Verify with numpy's solver
x_np = np.linalg.solve(A,b)
print("Check with np.linalg.solve:", np.allclose(x, x_np))


## 6) Do not invert to solve — compare `solve` vs explicit inverse

In [ ]:
rng = np.random.default_rng(0)
A = rng.normal(size=(5,5))
# Make A well-conditioned by symmetrizing and adding to diagonal
A = 0.5*(A + A.T) + 5*np.eye(5)
b = rng.normal(size=5)

x_solve = np.linalg.solve(A,b)
x_inv = np.linalg.inv(A) @ b

print("||A x_solve - b|| =", np.linalg.norm(A@x_solve - b))  # small
print("||A x_inv   - b|| =", np.linalg.norm(A@x_inv   - b))  # similar, but solve is preferred numerically and for speed


## 7) Conditioning — Vandermonde matrices get ill-conditioned quickly

In [ ]:
# Vandermonde appears in polynomial interpolation/fitting: A_ij = x_i^{n-j}
def vander(x, deg):
    # columns: [x^0, x^1, ..., x^deg]
    cols = [x**k for k in range(deg+1)]
    return np.vstack(cols).T

xs = np.linspace(-1, 1, 20)
degrees = np.arange(1, 16)
kappas = []
for d in degrees:
    A = vander(xs, d)
    print(A.shape)
    kappas.append(np.linalg.cond(A))

plt.figure()
plt.semilogy(degrees, kappas, 'o-')
plt.xlabel("polynomial degree")
plt.ylabel("cond(A) (2-norm, log scale)")
plt.title("Vandermonde matrices are often ill-conditioned")
plt.grid(True, which='both'); plt.tight_layout(); plt.show()


## 8) Multiple right-hand sides — same physics matrix, many load cases

In [ ]:
# Reuse the 1D conduction matrix A (interior nodes only) but with different boundary conditions.
# We can solve A X = B, where columns of B are different b vectors for different (T_left, T_right).

def build_conduction_matrix(n):
    A = np.zeros((n,n))
    for j in range(n):
        A[j,j] = -2.0
        if j-1 >= 0: A[j,j-1] = 1.0
        if j+1 < n: A[j,j+1] = 1.0
    return A

N_int = 20
A = build_conduction_matrix(N_int)

cases = [(80.0, 20.0), (100.0, 50.0), (50.0, 10.0)]  # (T_left, T_right)
B = np.zeros((N_int, len(cases)))
for k,(Tl, Tr) in enumerate(cases):
    b = np.zeros(N_int); b[0] -= Tl; b[-1] -= Tr
    B[:,k] = b

X = np.linalg.solve(A, B)  # columns are solutions for each case -- LU decomposition under the hood

plt.figure()
for k,(Tl, Tr) in enumerate(cases):
    T_full = np.r_[Tl, X[:,k], Tr]
    x_full = np.r_[0.0, x, 1.0]
    plt.plot(x_full, T_full, label=f"T_left={Tl}, T_right={Tr}")
plt.xlabel("x"); plt.ylabel("Temperature [°C]")
plt.title("Multiple boundary conditions solved with one factorization")
plt.legend(); plt.tight_layout(); plt.show()


## 9) Least squares — estimate gravitational acceleration from noisy height–time data

In [ ]:
# Physics model: y(t) = h0 + v0 t - (1/2) g t^2  (vertical position; upward positive)
# Unknown parameters: h0, v0, g. The model is linear in [h0, v0, (-1/2)g].
# Build design matrix A = [1, t, -t^2/2] and solve min ||A theta - y||_2.

rng = np.random.default_rng(10)
g_true = 9.81; h0_true = 1.4; v0_true = 5.0
t = np.linspace(0, 1.1, 40)
y_true = h0_true + v0_true*t - 0.5*g_true*t**2

sigma = 0.05                                  # measurement noise [m]
y_meas = y_true + rng.normal(0, sigma, size=t.size)

A = np.c_[np.ones_like(t), t, -0.5*t**2]
theta, residuals, rank, s = np.linalg.lstsq(A, y_meas, rcond=None)
h0_hat, v0_hat, g_hat = theta

print(f"Estimated h0={h0_hat:.3f} m, v0={v0_hat:.3f} m/s, g={g_hat:.3f} m/s^2  (true g={g_true})")
y_fit = A @ theta
rmse = np.sqrt(np.mean((y_meas - y_fit)**2))
print("RMSE =", round(float(rmse), 4), "m")

plt.figure()
plt.scatter(t, y_meas, s=18, label="measurements")
plt.plot(t, y_true, 'k--', label="true trajectory")
plt.plot(t, y_fit, 'r', lw=2, label="least-squares fit")
plt.xlabel("time t [s]"); plt.ylabel("height y [m]")
plt.title("Estimate g from noisy trajectory via linear least squares")
plt.legend(); plt.tight_layout(); plt.show()

plt.figure()
plt.plot(t, y_meas - y_fit, 'o-')
plt.axhline(0, color='k', lw=0.8)
plt.xlabel("time t [s]"); plt.ylabel("residual [m]")
plt.title("Fit residuals")
plt.tight_layout(); plt.show()


## 10) (Optional) Weighted least squares — down-weight worse sensors 

In [ ]:
# Suppose some measurements are less reliable (e.g., distant camera frames are blurrier).
# We can weight each sample by 1/sigma_i^2 by scaling rows of A and b.

rng = np.random.default_rng(2)
g_true = 9.81; h0_true = 2.0; v0_true = 3.5
t = np.linspace(0, 1.0, 35)
y_true = h0_true + v0_true*t - 0.5*g_true*t**2

sigma = 0.02 + 0.15*(t/t.max())              # noise grows with time (worse tracking later)
noise = rng.normal(0, sigma, size=t.size)
y_meas = y_true + noise

A = np.c_[np.ones_like(t), t, -0.5*t**2]

# Unweighted fit
theta_u, *_ = np.linalg.lstsq(A, y_meas, rcond=None)
g_u = theta_u[2]

# Weighted fit: scale rows by w_i = 1/sigma_i
w = 1.0/np.maximum(sigma, 1e-8)
Aw = A * w[:,None]
bw = y_meas * w
theta_w, *_ = np.linalg.lstsq(Aw, bw, rcond=None)
g_w = theta_w[2]

print(f"Unweighted g_hat={g_u:.3f}  |  Weighted g_hat={g_w:.3f}  |  true g={g_true}")
y_u = A @ theta_u; y_w = A @ theta_w

plt.figure()
plt.errorbar(t, y_meas, yerr=sigma, fmt='o', capsize=3, label="data (heteroskedastic)")
plt.plot(t, y_true, 'k--', label="true")
plt.plot(t, y_u, 'C1', label=f"unweighted fit (g={g_u:.2f})")
plt.plot(t, y_w, 'C3', label=f"weighted fit (g={g_w:.2f})")
plt.xlabel("time [s]"); plt.ylabel("height [m]"); plt.legend(); plt.tight_layout(); plt.show()
